# A Data Science Approach of Similarities and Differences Between Chicago Communties and Their Relationship with Community Average Income

## Part 1: Introduction/Business Problem

First let's consider this question. Imagin in some city, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office? 

By first looking at it, this could be a very complicated problem. One needs to do a lot of research, get familiar with the city, find relavant informations online, drive around ask local people, etc. But what if you want an easier method to get it done, which can make sure you don't miss a spot? This is where data science kicks in. 

In this project, we will take the city of Chicago as an example. We will first get all the community areas and neighborhoods in the city of Chicago, then we will get the coordinates of them. After that, we will use the API from Foursquare to extract the most visited venues in each community area. Then we will use unsupervised machine learning method - k-means to devide the communities into several clusters. You will see the similarities and differences between then. Last but not least, we will do some analysis on the data, and use the income data from Chicago Data Portal to help us figure out the deep relatinship between popular venues and communities' characters. 

## Part 2: Get Our Dataset No. 1

In this part, I will use the Foursquare location data to solve the problem and execute my idea. I will also use other datasets in combination with the Foursquare location data, which will be explained in the next part.

In [3]:
# Import all necessary libraries
import io, time, json
import requests
from bs4 import BeautifulSoup
import csv
import datetime
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

# !conda install -c conda-forge geocoder --yes
import geocoder # import geocoder
# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#### First we need to get all neighborhoods in Chicago. Let'suse BeautifulSoup to scrape data from wiki page. 

In [6]:
# Use BeautifulSoup to scrape data from wiki page
Area = []
Neighborhood = []
url = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.find('table')
rows = table.find_all('tr')
for row in rows[1:]:
    items = row.find_all('td')
    Area.append(items[1].text[:-1])
    Neighborhood.append(items[0].text[:-1])
df_original = pd.DataFrame({'CommunityArea': Area, 'Neighborhood': Neighborhood})
df_original.head(10)

,CommunityArea,Neighborhood
0,Albany Park,Albany Park
1,Riverdale,Altgeld Gardens
2,Edgewater,Andersonville
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square
5,Ashburn,Ashburn
6,Ashburn,Ashburn Estates
7,Auburn Gresham,Auburn Gresham
8,Avalon Park,Avalon Park
9,Avondale,Avondale


In [7]:
df_original.shape

(246, 2)

#### There are too many neighborhoods. So I decided to group the neighborhoods based on their community area. 

In [8]:
df_combined = df_original.groupby(['CommunityArea'])['Neighborhood'].apply(', '.join).reset_index().sort_values('CommunityArea')
df_combined.head(10)

,CommunityArea,Neighborhood
0,Albany Park,"Albany Park, Mayfair, North Mayfair, Ravenswoo..."
1,Archer Heights,Archer Heights
2,Armour Square,"Armour Square, Chinatown, Wentworth Gardens"
3,Ashburn,"Ashburn, Ashburn Estates, Beverly View, Crestl..."
4,Auburn Gresham,"Auburn Gresham, Gresham"
5,Austin,"Galewood, The Island, North Austin, South Austin"
6,"Austin, Humboldt Park",West Humboldt Park
7,Avalon Park,"Avalon Park, Marynook, Stony Island Park"
8,Avondale,"Avondale, Jackowo, Wacławowo"
9,"Avondale, Irving Park",Polish Village


In [9]:
df_combined.shape

(82, 2)

#### Now the number of rows are reduced down to 82. The next thing is to get the coordinates of all these community areas. 

In [10]:
def get_coordinates(CommunityArea):
    address = '{}, Chicago, IL'.format(CommunityArea)
    geolocator = Nominatim(user_agent="chicago_explorer")
    location = geolocator.geocode(address)
    if location is not None:
        return location.latitude, location.longitude
    else: 
        return 'NaN', 'NaN'
for i, CommunityArea in enumerate(list(df_combined['CommunityArea'])): 
    df_combined.ix[i,'Latitude'], df_combined.ix[i,'Longitude'] = get_coordinates(CommunityArea)
df_combined.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,CommunityArea,Neighborhood,Latitude,Longitude
0,Albany Park,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",41.9719,-87.7162
1,Archer Heights,Archer Heights,41.8114,-87.7262
2,Armour Square,"Armour Square, Chinatown, Wentworth Gardens",41.84,-87.6331
3,Ashburn,"Ashburn, Ashburn Estates, Beverly View, Crestl...",41.7475,-87.7112
4,Auburn Gresham,"Auburn Gresham, Gresham",41.7505,-87.6643
5,Austin,"Galewood, The Island, North Austin, South Austin",41.8879,-87.7649
6,"Austin, Humboldt Park",West Humboldt Park,NaN,NaN
7,Avalon Park,"Avalon Park, Marynook, Stony Island Park",41.745,-87.5887
8,Avondale,"Avondale, Jackowo, Wacławowo",41.9389,-87.7112
9,"Avondale, Irving Park",Polish Village,41.9534,-87.7364


#### We noticed that some coordinates are missing in the dataframe. Let's get them and see how many are there. 

In [11]:
df_invalid = df_combined[df_combined['Latitude']=='NaN']
df_invalid

,CommunityArea,Neighborhood,Latitude,Longitude
6,"Austin, Humboldt Park",West Humboldt Park,NaN,NaN
17,"Chatham, Roseland",West Chesterfield,NaN,NaN
80,"West Town, Logan Square",Polish Downtown,NaN,NaN


#### There are only three community areas missing coordinates. So we manually add their coordinates by simply searching them in google. 
#### There is one more, which has a index 66, has the wrong coordinate. I actually found this by the folium map. But I came back here to changed its coordinates to the correct one. 

In [12]:
df_combined.ix[6,'Latitude'], df_combined.ix[6,'Longitude'] = 41.9025, -87.7361
df_combined.ix[17,'Latitude'], df_combined.ix[17,'Longitude'] = 41.7291, -87.6192
df_combined.ix[80,'Latitude'], df_combined.ix[80,'Longitude'] = 41.9231, -87.7093
df_combined.ix[66,'Latitude'], df_combined.ix[66,'Longitude'] = 41.71, -87.56
df_combined.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_i

,CommunityArea,Neighborhood,Latitude,Longitude
0,Albany Park,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",41.9719,-87.7162
1,Archer Heights,Archer Heights,41.8114,-87.7262
2,Armour Square,"Armour Square, Chinatown, Wentworth Gardens",41.84,-87.6331
3,Ashburn,"Ashburn, Ashburn Estates, Beverly View, Crestl...",41.7475,-87.7112
4,Auburn Gresham,"Auburn Gresham, Gresham",41.7505,-87.6643
5,Austin,"Galewood, The Island, North Austin, South Austin",41.8879,-87.7649
6,"Austin, Humboldt Park",West Humboldt Park,41.9025,-87.7361
7,Avalon Park,"Avalon Park, Marynook, Stony Island Park",41.745,-87.5887
8,Avondale,"Avondale, Jackowo, Wacławowo",41.9389,-87.7112
9,"Avondale, Irving Park",Polish Village,41.9534,-87.7364


#### Now our data is ready! Let's first save it to csv file for future use. 

In [13]:
df_combined.to_csv("chicago_neighborhoods.csv")

#### Let's get the coordinates of Chicago. 

In [14]:
address = 'Chicago, Illinois'
geolocator = Nominatim(user_agent="chicago_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 41.8755616, -87.6244212.


#### Let's how a map of Chicago with all communites. 

In [15]:
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_combined['Latitude'], df_combined['Longitude'], df_combined['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

#### Now we are ready to get all venues near each community. Here we use Foursquare database. 

In [44]:
CLIENT_ID = '5V420ZYIXEK0RKAKR4OI5QAA4LRTGTOPC4LILILNARRWN05U' # your Foursquare ID
CLIENT_SECRET = 'F2GJOF2BBLR0KSPIUOE1NWL1RW1GJG3ONXNXT5ZAEXLIPTC3' # your Foursquare Secret
VERSION = '20190808' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
chicago_venues = getNearbyVenues(names=df_combined['Neighborhood'],
                                   latitudes=df_combined['Latitude'],
                                   longitudes=df_combined['Longitude']
                                  )

Albany Park, Mayfair, North Mayfair, Ravenswood Manor
Archer Heights
Armour Square, Chinatown, Wentworth Gardens
Ashburn, Ashburn Estates, Beverly View, Crestline, Parkview, Scottsdale, Wrightwood
Auburn Gresham, Gresham
Galewood, The Island, North Austin, South Austin
West Humboldt Park
Avalon Park, Marynook, Stony Island Park
Avondale, Jackowo, Wacławowo
Polish Village
Belmont Central, Brickyard, Cragin, Hanson Park
Beverly, East Beverly, West Beverly
Bridgeport
Brighton Park
Burnside
Calumet Heights, Pill Hill
Chatham, East Chatham, West Chatham
West Chesterfield
Chicago Lawn, Lithuanian Plaza, Marquette Park
Chrysler Village, Clearing East, Clearing West
Bronzeville, Dearborn Homes, The Gap, Groveland Park, Lake Meadows, Prairie Shores, South Commons, Stateway Gardens
Belmont Heights, Belmont Terrace, Dunning, Irving Woods, Schorsch Village
East Garfield Park, Fifth City
East Side
Andersonville, Edgewater, Edgewater Beach, Edgewater Glen, Lakewood / Balmoral
Edison Park
Englewood, 

In [47]:
# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chicago_onehot['Neighborhood'] = chicago_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

# chicago_onehot.head()
chicago_grouped = chicago_onehot.groupby('Neighborhood').mean().reset_index()

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = chicago_grouped['Neighborhood']

for ind in np.arange(chicago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",Sandwich Place,Pizza Place,Donut Shop,Fried Chicken Joint,Bakery,Mobile Phone Shop,Gas Station,Diner,Chinese Restaurant,Bus Station
1,"Andersonville, Edgewater, Edgewater Beach, Edg...",Bank,Asian Restaurant,Sandwich Place,Sushi Restaurant,Mexican Restaurant,Coffee Shop,Pharmacy,Antique Shop,Video Store,Mobile Phone Shop
2,Archer Heights,Mexican Restaurant,Mobile Phone Shop,Grocery Store,Gas Station,Optical Shop,Candy Store,Big Box Store,Bar,Bank,Bakery
3,"Armour Square, Chinatown, Wentworth Gardens",Chinese Restaurant,Sports Bar,Hot Dog Joint,Breakfast Spot,Gas Station,Italian Restaurant,Asian Restaurant,Indian Restaurant,Sandwich Place,Grocery Store
4,"Ashburn, Ashburn Estates, Beverly View, Crestl...",Construction & Landscaping,Cosmetics Shop,Light Rail Station,Italian Restaurant,Automotive Shop,Cuban Restaurant,Currency Exchange,Fish Market,Fish & Chips Shop,Filipino Restaurant


## Part 3: Analyzing Data with Machine Learning

#### Here we start our analyzing part. We first try k-means to cluster the communites into 5 clusters and see if there is anything interesting. 

In [49]:
kclusters = 5

chicago_grouped_clustering = chicago_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [50]:
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 4, 3, 3, 2, 3, 3, 3, 3, 0, 3,
       3, 3, 3, 3, 2, 3, 2, 0, 3, 3, 1, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3,
       3, 0, 3, 0, 3, 3, 2, 0, 2, 2, 3, 3, 3, 0, 0, 3, 0, 2, 3, 3, 3, 2,
       3, 3, 3, 3, 0, 3, 3, 3, 2, 0, 3, 3, 2, 3, 2], dtype=int32)

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chicago_merged = df_combined

# merge chicago_grouped with chicago_data to add latitude/longitude for each neighborhood
chicago_merged = chicago_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
chicago_merged.head() # check the last columns!

,CommunityArea,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",41.9719,-87.7162,3.0,Sandwich Place,Pizza Place,Donut Shop,Fried Chicken Joint,Bakery,Mobile Phone Shop,Gas Station,Diner,Chinese Restaurant,Bus Station
1,Archer Heights,Archer Heights,41.8114,-87.7262,3.0,Mexican Restaurant,Mobile Phone Shop,Grocery Store,Gas Station,Optical Shop,Candy Store,Big Box Store,Bar,Bank,Bakery
2,Armour Square,"Armour Square, Chinatown, Wentworth Gardens",41.84,-87.6331,3.0,Chinese Restaurant,Sports Bar,Hot Dog Joint,Breakfast Spot,Gas Station,Italian Restaurant,Asian Restaurant,Indian Restaurant,Sandwich Place,Grocery Store
3,Ashburn,"Ashburn, Ashburn Estates, Beverly View, Crestl...",41.7475,-87.7112,3.0,Construction & Landscaping,Cosmetics Shop,Light Rail Station,Italian Restaurant,Automotive Shop,Cuban Restaurant,Currency Exchange,Fish Market,Fish & Chips Shop,Filipino Restaurant
4,Auburn Gresham,"Auburn Gresham, Gresham",41.7505,-87.6643,3.0,Fast Food Restaurant,Lounge,Greek Restaurant,Pharmacy,Cosmetics Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Eye Doctor,Fabric Shop


In [52]:
# Check if there is any community with zero venues
df_null = chicago_merged[chicago_merged['1st Most Common Venue'].isnull()]
df_null

,CommunityArea,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Riverdale,"Altgeld Gardens, Eden Green, Golden Gate, Rive...",41.6678,-87.605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Delete it
chicago_merged = chicago_merged.dropna()
chicago_merged.head()

,CommunityArea,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",41.9719,-87.7162,3.0,Sandwich Place,Pizza Place,Donut Shop,Fried Chicken Joint,Bakery,Mobile Phone Shop,Gas Station,Diner,Chinese Restaurant,Bus Station
1,Archer Heights,Archer Heights,41.8114,-87.7262,3.0,Mexican Restaurant,Mobile Phone Shop,Grocery Store,Gas Station,Optical Shop,Candy Store,Big Box Store,Bar,Bank,Bakery
2,Armour Square,"Armour Square, Chinatown, Wentworth Gardens",41.84,-87.6331,3.0,Chinese Restaurant,Sports Bar,Hot Dog Joint,Breakfast Spot,Gas Station,Italian Restaurant,Asian Restaurant,Indian Restaurant,Sandwich Place,Grocery Store
3,Ashburn,"Ashburn, Ashburn Estates, Beverly View, Crestl...",41.7475,-87.7112,3.0,Construction & Landscaping,Cosmetics Shop,Light Rail Station,Italian Restaurant,Automotive Shop,Cuban Restaurant,Currency Exchange,Fish Market,Fish & Chips Shop,Filipino Restaurant
4,Auburn Gresham,"Auburn Gresham, Gresham",41.7505,-87.6643,3.0,Fast Food Restaurant,Lounge,Greek Restaurant,Pharmacy,Cosmetics Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Eye Doctor,Fabric Shop


In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['Neighborhood'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Chicago is an interesting city because it is distributed evenly from the very downtown to suburb. Its structure is so clear which makes one admire the beauty of the city. The structure is reflected through the clusters of communities. Where you can clearly see the green clusters are closter to the downtown, while red and blue clusters are in the suburb area. 

## Part 4: Get Our Dataset No.2

#### What's more interesting is, based on my knowledge of the the city of Chicago, the red clusters are relatively poor residential communites, while the other clusters like green (closest to downtown) and blue are residential communites with above average income families. 

#### It is hard to make that statement without data. So let's look at the data. 

#### I got the data from [Chicago Data Portal](https://data.cityofchicago.org/Health-Human-Services/Per-Capita-Income/r6ad-wvtk). This data is imported and shown below. 

In [59]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_ab964ce83af04598aba0ceae8b8cebd1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vsCHOwNGE2e-xGOALgBEGQMiaGij-pifQK8ZU2gJ_bxN',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ab964ce83af04598aba0ceae8b8cebd1.get_object(Bucket='assignmentjypyter-donotdelete-pr-bkiijlr75tmfgv',Key='Per_Capita_Income.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_income = pd.read_csv(body)
df_income.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


#### For our purpose, we only care about the income of every community. So we will keep only two columns: 'COMMUNITY AREA NAME' and 'PER CAPITA INCOME'.  

In [65]:
df_income_chopped = df_income[['COMMUNITY AREA NAME', 'PER CAPITA INCOME ']]
df_income_chopped = df_income_chopped.rename(columns={"COMMUNITY AREA NAME": "CommunityArea", 
                                  "PER CAPITA INCOME ": 'Income'})
df_income_chopped.head()

,CommunityArea,Income
0,Rogers Park,23939
1,West Ridge,23040
2,Uptown,35787
3,Lincoln Square,37524
4,North Center,57123


In [67]:
df_income_chopped.shape

(78, 2)

In [68]:
chicago_merged.shape

(81, 15)

#### As you can see, the new data we got from Chicago Data Portal only has 78 communites, while the orignal one we got from wiki has 81. This is okay since we will draw our conclusions based on clusters, not individual communty. 

#### Now we merge the two data frame into one, using inner join. All unmatched communites will be discarded. 

In [69]:
df_merged = chicago_merged.merge(df_income_chopped, how = 'inner', on = ['CommunityArea'])
df_merged.head()

,CommunityArea,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Income
0,Albany Park,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",41.9719,-87.7162,3.0,Sandwich Place,Pizza Place,Donut Shop,Fried Chicken Joint,Bakery,Mobile Phone Shop,Gas Station,Diner,Chinese Restaurant,Bus Station,21323
1,Archer Heights,Archer Heights,41.8114,-87.7262,3.0,Mexican Restaurant,Mobile Phone Shop,Grocery Store,Gas Station,Optical Shop,Candy Store,Big Box Store,Bar,Bank,Bakery,16134
2,Armour Square,"Armour Square, Chinatown, Wentworth Gardens",41.84,-87.6331,3.0,Chinese Restaurant,Sports Bar,Hot Dog Joint,Breakfast Spot,Gas Station,Italian Restaurant,Asian Restaurant,Indian Restaurant,Sandwich Place,Grocery Store,16148
3,Ashburn,"Ashburn, Ashburn Estates, Beverly View, Crestl...",41.7475,-87.7112,3.0,Construction & Landscaping,Cosmetics Shop,Light Rail Station,Italian Restaurant,Automotive Shop,Cuban Restaurant,Currency Exchange,Fish Market,Fish & Chips Shop,Filipino Restaurant,23482
4,Auburn Gresham,"Auburn Gresham, Gresham",41.7505,-87.6643,3.0,Fast Food Restaurant,Lounge,Greek Restaurant,Pharmacy,Cosmetics Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Eye Doctor,Fabric Shop,15528


In [70]:
df_merged.shape

(72, 16)

#### So after merging the two dataframe, we have 72 communites left.  

#### Now we use groupby to get the average Income of each cluster. Let's also add the color of each cluster. 

In [78]:
df_cluster_income = df_merged.groupby(['Cluster Labels']).mean()
df_cluster_income['Color'] = ['Red', 'Purple', 'Blue', 'Green', 'Brown']
df_cluster_income.head()

,Income,Color
Cluster Labels,,
0.0,16776.615385,Red
1.0,12034.000000,Purple
2.0,24601.125000,Blue
3.0,28283.836735,Green
4.0,26282.000000,Brown


## Part 5: Discussions

So the dataframe shown above agrees with my  ealier assumptions. The red clusters are relatively poor residential communites, while the other clusters like green (closest to downtown) and blue are residential communites with above average income families. The Brown cluster only has one community, it is most likely very rich community. The Purple cluster only has one community, it is most likely very poor community. 

#### This is very interesting because we got the clusters from venues near each community. How would the venues near each community correlated to average income? Are centain stores/restaurant open more often in rich communities or vise versa? 


Let's first see how many communities are in each cluster. 

In [77]:
df_cluster_income_2 = df_merged.groupby(['Cluster Labels']).count()
df_cluster_income_2

,CommunityArea,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Income
Cluster Labels,,,,,,,,,,,,,,,
0.0,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2.0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
3.0,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49
4.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


#### Then what I am going to do is to find the most three counted venues in each cluster. 

In [98]:
df_merged_copy = df_merged
df_merged_copy["Most Common Count"] = (df_merged_copy.groupby('Cluster Labels')["1st Most Common Venue"]
                        .transform(lambda x: x.value_counts().index[0]))
df_merged_copy = df_merged_copy[['Cluster Labels', "Most Common Count"]].groupby('Cluster Labels')
df_merged_copy.head()

,Cluster Labels,Most Common Count
0,3.0,Fast Food Restaurant
1,3.0,Fast Food Restaurant
2,3.0,Fast Food Restaurant
3,3.0,Fast Food Restaurant
4,3.0,Fast Food Restaurant
8,0.0,Mexican Restaurant
11,0.0,Mexican Restaurant
13,2.0,Bus Station
14,2.0,Bus Station
15,0.0,Mexican Restaurant


It looks weird that there are multiple rows for the same cluster. But we can still see the most common venues for each cluster. So let's add it to the ealier dataframe. 

In [100]:
df_cluster_income['Most Common Venue'] = \
['Mexican Restaurant', 'Convenience Store', 'Bus Station', 
 'Fast Food Restaurant', 'Deli / Bodega']
df_cluster_income.head()

,Income,Color,Most Common Venue
Cluster Labels,,,
0.0,16776.615385,Red,Mexican Restaurant
1.0,12034.000000,Purple,Convenience Store
2.0,24601.125000,Blue,Bus Station
3.0,28283.836735,Green,Fast Food Restaurant
4.0,26282.000000,Brown,Deli / Bodega


#### Let's repeat it for the 2nd and 3rd most common count

In [105]:
df_merged.columns
df_merged_copy = df_merged
df_merged_copy["2nd Most Common Count"] = (df_merged_copy.groupby('Cluster Labels')["2nd Most Common Venue"]
                        .transform(lambda x: x.value_counts().index[0]))
df_merged_copy = df_merged_copy[['Cluster Labels', "2nd Most Common Count"]].groupby('Cluster Labels')
df_merged_copy.head()

,Cluster Labels,2nd Most Common Count
0,3.0,Pizza Place
1,3.0,Pizza Place
2,3.0,Pizza Place
3,3.0,Pizza Place
4,3.0,Pizza Place
8,0.0,Mexican Restaurant
11,0.0,Mexican Restaurant
13,2.0,Gymnastics Gym
14,2.0,Gymnastics Gym
15,0.0,Mexican Restaurant


In [106]:
df_merged.columns
df_merged_copy = df_merged
df_merged_copy["3rd Most Common Count"] = (df_merged_copy.groupby('Cluster Labels')["3rd Most Common Venue"]
                        .transform(lambda x: x.value_counts().index[0]))
df_merged_copy = df_merged_copy[['Cluster Labels', "3rd Most Common Count"]].groupby('Cluster Labels')
df_merged_copy.head()

,Cluster Labels,3rd Most Common Count
0,3.0,Train Station
1,3.0,Train Station
2,3.0,Train Station
3,3.0,Train Station
4,3.0,Train Station
8,0.0,American Restaurant
11,0.0,American Restaurant
13,2.0,Park
14,2.0,Park
15,0.0,American Restaurant


In [107]:
df_cluster_income['2nd Most Common Venue'] = \
['Mexican Restaurant', 'Yoga Studio', 'Gymnastics Gym', 
 'Pizza Place', 'Bar']
df_cluster_income['3rd Most Common Venue'] = \
['American Restaurant', 'Donut Shop', 'Park', 
 'Train Station', 'Yoga Studio']
df_cluster_income.head()

,Income,Color,Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
Cluster Labels,,,,,
0.0,16776.615385,Red,Mexican Restaurant,Mexican Restaurant,American Restaurant
1.0,12034.000000,Purple,Convenience Store,Yoga Studio,Donut Shop
2.0,24601.125000,Blue,Bus Station,Gymnastics Gym,Park
3.0,28283.836735,Green,Fast Food Restaurant,Pizza Place,Train Station
4.0,26282.000000,Brown,Deli / Bodega,Bar,Yoga Studio


In the above dataframe, we can see that the cluster 0, which is a below average income community cluster, has a lot of Mexican Restaurant. This indites that these areas, which are most likely Mexican resident areas, are relatively poor.

Cluster 1 only has one community in there, so it may not represent too much. It has Convenience Store in the first place and Yoga Studio at the second place. Notice that Yoga Studio also exist as the third place in cluster 4, which also only has one community.

In cluster 3, which is near downtown area, we see a lot of Fast Food Restaurant and Pizza Place, that's because this is a area mainly lived by single people who live in expensive condos and work in downtown area. They either don't have a kitchen in the apartment or don't have time to cook. So fast food restaurants and pizza places are polular. 

Blue Cluster 2 is close to downtown but not that close. This is the area where above average income people with family live in. Many of them still work in the downtown center, but they buy houses in the area where it is cheaper than downtown so they can buy a bigger house. They still need public transportations to go to work, that explains why Bus Station is very common in these area. A lot of Gymnastics Gym indicates the people live here live more healthy and maybe happier. There are many parks in this area, which indicates this is a good area to live in. 

So now we can help people who is looking to open a restaurant, and who is trying to start their own business. For example, if someone want to start a Mexican Restaurant, you'd better do it in one of the communites in cluster 0. But try to avoid the communites with too many Mexican Restaurant, becasue there may be a lot of competation. And if someone wants to open a gym, we could recommend to do it in one of the communites in cluster 2 because the people in this area are more likely to visit a gym from time to time. 


## Thanks for reading this project!